In [1]:
import numba as nb
import numpy as np
from itertools import repeat

In [2]:
@nb.njit(nb.void(nb.int64, nb.int64))
def inf_sequence(a,b):
    num = a
    while True:
        yield num
        num += b

In [3]:
@nb.njit(nb.void(nb.int64))
def repeat(a):
    while True:
        yield a

In [5]:
@nb.njit(nb.int64(nb.int64))
def test(x):
    a=inf_sequence(10,2)
    b=repeat(10)
    for ii,jj,kk in zip(range(3),a,b):
        print(ii, jj, kk)
    return x

In [ ]:
@nb.njit(nb.float64[:](nb.float64[:,:],nb.float64[:]))
def matmul(m,v):
    o = np.zeros(m.shape[0], nb.float64)
    for ii in range(m.shape[0]):
        for jj in range(m.shape[1]):
            o[ii]+=m[ii,jj]*v[jj]
    return o

In [ ]:
@nb.njit(nb.float64[:](nb.float64[:,:],nb.float64[:]))
def transpmatmul(m,v):
    o = np.zeros(m.shape[1], nb.float64)
    for jj in range(m.shape[0]):
        for ii in range(m.shape[1]):
            o[ii]+=m[jj,ii]*v[jj]
    return o

In [ ]:
m=np.ones((3,3))
m[1,0]=0.0
m[2,0]=0.0
m[2,1]=0.0
m

In [ ]:
v=np.ones(3)

In [ ]:
transpmatmul(m,v)

In [6]:
test(10)

0 10 10
1 12 10
2 14 10


10